In [88]:
# %pip freeze > requirements.txt
# %pip install -r requirement.txt

In [89]:
from bs4 import BeautifulSoup
from datetime import datetime

from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import json
import pandas as pd
import pymysql
import re
import requests
import sqlite3
import time
import urllib.parse

# logging.basicConfig(filename='error.log', level=logging.ERROR)

In [90]:
def clean_table_name(jenis, filter_wilayah=''):
    propinsi = filter_wilayah['PROPINSI'].replace(' ','').lower()
    kota = filter_wilayah['KOTA'].replace(' ','').lower()
    kecamatan = filter_wilayah['KECAMATAN'].replace(' ','').lower()
    kelurahan = filter_wilayah['KELURAHAN'].replace(' ','').lower()

    jenis_table = jenis.replace(' ', '')

    if propinsi:
        jenis_table += f'_{propinsi}'
    if kota:
        jenis_table += f'_{kota}'
    if kecamatan:
        jenis_table += f'_{kecamatan}'
    if kelurahan:
        jenis_table += f'_{kelurahan}'
        
    return jenis_table

In [91]:
def db_check(database, table_name):
    if database.lower() == 'sqlite':
        with sqlite3.connect("./backend/data.db") as connection:
            cursor = connection.cursor()
            tables = {
                f"{table_name}": '"ID" INTEGER PRIMARY KEY NOT NULL, "NAMA" TEXT, "KOORDINAT" TEXT, "JML_RATING" INTEGER, "ALAMAT" TEXT, "TAG_GOOGLE" TEXT, "KELURAHAN" TEXT, "KECAMATAN" TEXT, "KOTA" TEXT, "PROVINSI" TEXT, "TIPE" TEXT, "IDCARI" INTEGER, "DATA_UPDATE" DATETIME',
                "randomized_pos": '"ID" INTEGER PRIMARY KEY NOT NULL, "PROPINSI" TEXT, "KOTA" TEXT, "KECAMATAN" TEXT, "KELURAHAN" TEXT, "KODEPOS" TEXT, "DATA_UPDATE"'
            }
            for table, schema in tables.items():
                cursor.execute(f'CREATE TABLE IF NOT EXISTS {table} ({schema})')

    elif database.lower() == 'mariadb':
        host, port, user, password, database = [i.replace(' ','') for i in open('authentication/mariadb', 'r').read().split(',')]
        connection = pymysql.connect(host=host, port=int(port), user=user, password=password, database=database, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

        try:
            with connection.cursor() as cursor:
                tables = {
                    f'{table_name}': 'ID INT AUTO_INCREMENT PRIMARY KEY, NAMA TEXT, KOORDINAT TEXT, JML_RATING INT, ALAMAT TEXT, TAG_GOOGLE TEXT, KELURAHAN TEXT, KECAMATAN TEXT, KOTA TEXT, PROVINSI TEXT, TIPE TEXT, IDCARI INT, DATA_UPDATE DATETIME',
                    'randomized_pos': 'ID INT AUTO_INCREMENT PRIMARY KEY, PROPINSI TEXT, KOTA TEXT, KECAMATAN TEXT, KELURAHAN TEXT, KODEPOS TEXT, DATA_UPDATE DATETIME'
                }
                for table, schema in tables.items():
                    cursor.execute(f'CREATE TABLE IF NOT EXISTS {table} ({schema})')
            connection.commit()
        finally:
            connection.close()

    else:
        print('Database tidak dikenal')

In [92]:
# df_pos = pd.read_csv('../scrape_kode_pos_indonesia/output/kode_pos.csv')
# df_pos = df_pos.fillna('-')
# df_cari = pd.DataFrame(df_pos['KOTA'].unique(), columns=['KOTA'])

# cek database, kalau kosong isi randomized
def random_pos_check(database):
    df_cari = pd.read_csv('../scrape_kode_pos_indonesia/output/kode_pos.csv', dtype=str)
    df_cari = df_cari.sample(frac=1).reset_index(drop=True) # randomized order
    df_cari.fillna('', inplace=True)

    values = []
    for i in range (len(df_cari)):
        propinsi = df_cari.iloc[i]['PROPINSI']
        kota = df_cari.iloc[i]['KOTA']
        kecamatan = df_cari.iloc[i]['KECAMATAN']
        kelurahan = df_cari.iloc[i]['KELURAHAN']
        kodepos = df_cari.iloc[i]['KODE POS']
        update_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        values.append((propinsi, kota, kecamatan, kelurahan, kodepos, update_time))

    if database.lower() == 'sqlite':
        with sqlite3.connect('./backend/data.db') as connection:
            cursor = connection.cursor()
            cursor.execute('SELECT COUNT(*) FROM randomized_pos')
            count = cursor.fetchone()[0]
            if count == 0:
                query = ('INSERT INTO randomized_pos (PROPINSI, KOTA, KECAMATAN, KELURAHAN, KODEPOS, DATA_UPDATE) VALUES (?, ?, ?, ?, ?, ?)')
                cursor.executemany(query, values)

    elif database.lower() == 'mariadb':
        host, port, user, password, database = [i.replace(' ','') for i in open('authentication/mariadb', 'r').read().split(',')]
        connection = pymysql.connect(host=host, port=int(port), user=user, password=password, database=database, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

        try:
            with connection.cursor() as cursor:
                cursor.execute('SELECT COUNT(*) FROM randomized_pos')
                count = cursor.fetchone()['COUNT(*)']
                if count == 0:                    
                    query = ('INSERT INTO randomized_pos (PROPINSI, KOTA, KECAMATAN, KELURAHAN, KODEPOS, DATA_UPDATE) VALUES (%s, %s, %s, %s, %s, %s)')                 
                    cursor.executemany(query, values)
                
            connection.commit()

        finally:
            connection.close()

    else:
        print('Database tidak dikenal')

In [93]:
def create_new_df_cari(database, jenis, filter_wilayah=''):
    table_name = clean_table_name(jenis, filter_wilayah)
    
    if database.lower() == 'sqlite':
        try:
            with sqlite3.connect('backend/data.db') as connection:
                cursor = connection.cursor()
                cursor.execute(f'SELECT IDCARI FROM {table_name} ORDER BY ID DESC LIMIT 1')
                last_cari = cursor.fetchone()[0]
        except Exception:
            last_cari = 0

        query = f'SELECT PROPINSI, KOTA, KECAMATAN, KELURAHAN, KODEPOS, ID AS IDCARI FROM randomized_pos WHERE IDCARI > {last_cari}'
        if filter_wilayah:
            propinsi = filter_wilayah['PROPINSI'].replace(' ','').lower()
            kota = filter_wilayah['KOTA'].replace(' ','').lower()
            kecamatan = filter_wilayah['KECAMATAN'].replace(' ','').lower()
            kelurahan = filter_wilayah['KELURAHAN'].replace(' ','').lower()

        if propinsi:
            query += f' AND PROPINSI = {propinsi}'
        if kota:
            query += f' AND KOTA = {kota}'
        if kecamatan:
            query += f' AND KECAMATAN = {kecamatan}'
        if kelurahan:
            query += f' AND KELURAHAN = {kelurahan}'

        with sqlite3.connect('backend/data.db') as connection:
            df_cari = pd.DataFrame(pd.read_sql_query(query, connection))
        
        return df_cari
    
    elif database.lower() == 'mariadb':
        host, port, user, password, database = [i.replace(' ','') for i in open('authentication/mariadb', 'r').read().split(',')]
        connection = pymysql.connect(host=host, port=int(port), user=user, password=password, database=database, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

        try:
            with connection.cursor() as cursor:
                try:
                    cursor.execute(f'SELECT IDCARI FROM {table_name} ORDER BY ID DESC LIMIT 1')
                    last_cari = cursor.fetchone()[0]
                except Exception:
                    last_cari = 0

                query = f'SELECT PROPINSI, KOTA, KECAMATAN, KELURAHAN, KODEPOS, ID AS IDCARI FROM randomized_pos WHERE IDCARI > {last_cari}'
                if filter_wilayah:
                    propinsi = filter_wilayah['PROPINSI'].replace(' ','').lower()
                    kota = filter_wilayah['KOTA'].replace(' ','').lower()
                    kecamatan = filter_wilayah['KECAMATAN'].replace(' ','').lower()
                    kelurahan = filter_wilayah['KELURAHAN'].replace(' ','').lower()

                if propinsi:
                    query += f' AND PROPINSI = {propinsi}'
                if kota:
                    query += f' AND KOTA = {kota}'
                if kecamatan:
                    query += f' AND KECAMATAN = {kecamatan}'
                if kelurahan:
                    query += f' AND KELURAHAN = {kelurahan}'

                df_cari = pd.DataFrame(pd.read_sql_query(query, connection))

        finally:
            connection.close()
            
        return df_cari

In [94]:
def remove_spaces(input_string):
    result_string = input_string.replace(" ", "")
    return result_string

def create_search_link(query: str, lang, geo_coordinates, zoom):
    if geo_coordinates is None and zoom is not None:
        raise ValueError("geo_coordinates must be provided along with zoom")

    endpoint = urllib.parse.quote_plus(query)

    params = {'authuser': '0',
              'hl': lang,
              'entry': 'ttu',} if lang is not None else {'authuser': '0',
                                                         'entry': 'ttu',}
    
    geo_str = ''
    if geo_coordinates is not None:
        geo_coordinates = remove_spaces(geo_coordinates)
        if zoom is not None:
            geo_str = f'/@{geo_coordinates},{zoom}z'
        else:
            geo_str = f'/@{geo_coordinates}'

    url = f'https://www.google.com/maps/search/{endpoint}'
    if geo_str:
        url += geo_str
    url += f'?{urllib.parse.urlencode(params)}'

    return url

In [95]:
def proxy_auth(proxy_name):
    user, password, domain = [i.replace(' ','') for i in open(f'authentication/{proxy_name}', 'r').read().split(',')]
    return user, password, domain

In [96]:
proxy_auth('proxyscrape')

('dl0kskmfsl8ssvi', 'x2z4c0y1fqnvm15', 'rp.proxyscrape.com:6060')

In [97]:
def map_scraper(database, jenis, filter_wilayah, proxy=''):
    if proxy:
        user, password, domain = proxy_auth('proxyscrape')
        proxy_insert = "{}:{}@{}".format(user, password, domain)

    df_cari = create_new_df_cari(database, jenis, filter_wilayah)

    for i in range(len(df_cari)):
        total_time = time.time()
        propinsi = df_cari.iloc[i].iloc[0]
        kota = df_cari.iloc[i].iloc[1]
        kecamatan = df_cari.iloc[i].iloc[2]
        kelurahan = df_cari.iloc[i].iloc[3]
        idcari = int(df_cari.iloc[i].iloc[5])
        cari = f'{jenis} in {kelurahan}, {kecamatan}, {kota}, {propinsi}'
        url_cari = create_search_link(cari, None, '', 18)
    
        if proxy:
            retry_count = 0
            while retry_count <= 60:
                try:
                    proxy_detail = {
                            "https":f"http://{proxy_insert}"
                        }
                    response = requests.get(url_cari, proxies=proxy_detail)
                    if response.status_code == 200:
                        break
                except Exception as e:
                    print(e)
                    print('Proxy gagal, mencoba proxy lain')
                    pass
        
                retry_count += 1
            
            if retry_count > 60:
                print('Seluruh proxy gagal')
                break
        else:
            response = requests.get(url_cari)
    
        data_cari = response.text
        soup_cari = BeautifulSoup(data_cari, 'html.parser')
        scripts = soup_cari.find_all('script')
    
        for script in scripts:
            if 'window.APP_INITIALIZATION_STATE' in str(script):
                data = str(script).split('=',3)[3]
                data2 = data.rsplit(';',10)[0]
                json_data = json.loads(data2)
                usaha = json_data[3][2][5:]
                json_usaha = json.loads(usaha)
                values = []
                a = 1
                while True:
                    try:
                        nama = json_usaha[0][1][a][14][11]
                        koordinat = ', '.join(list(map(str, json_usaha[0][1][a][14][9][-2:])))
                        alamat = ', '.join(json_usaha[0][1][a][14][2])
                        try:
                            rating = json_usaha[0][1][a][14][4][3][1]
                            index_of_space = rating.find(" ")
                            jml_rating = int(rating[:index_of_space])
                        except Exception:
                            jml_rating = 0
                        updatetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        values.append((nama, koordinat, jml_rating, alamat, kelurahan, kecamatan, kota, propinsi, jenis, idcari, updatetime))
                        a += 1

                    except Exception:
                        break

                query = f'INSERT INTO {clean_table_name(jenis, filter_wilayah)} (NAMA, KOORDINAT, JML_RATING, ALAMAT, KELURAHAN, KECAMATAN, KOTA, PROVINSI, TIPE, IDCARI, DATA_UPDATE) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
                
                if database.lower() == 'sqlite':
                    with sqlite3.connect('backend/data.db') as connection:
                        cursor = connection.cursor()
                        cursor.executemany(query, values)

                elif database.lower() == 'mariadb':
                    host, port, user, password, database = [i.replace(' ','') for i in open('authentication/mariadb', 'r').read().split(',')]
                    connection = pymysql.connect(host=host, port=int(port), user=user, password=password, database=database, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

                    try:
                        with connection.cursor() as cursor:
                            cursor = connection.cursor()
                            cursor.executemany(query, values)
                    finally:
                        connection.close()                     

                else:
                    print('Database tidak dikenal') 
                                        
        print(f'{jenis} di kelurahan {kelurahan} kecamatan {kecamatan} kota {kota} provinsi {propinsi} selesai diinput sebanyak {a-1} data')
        print(f'Total waktu query {time.time() - total_time}')
    
    print(f'Scrape {jenis} selesai')

In [98]:
def get_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    
    # proxyscrape.com
    username = "dl0kskmfsl8ssvi"
    password = "x2z4c0y1fqnvm15"
    proxy = "rp.proxyscrape.com:6060"
    proxy_auth = "{}:{}@{}".format(username, password, proxy)

    prox = Proxy()
    prox.proxy_type = ProxyType.MANUAL
    prox.ssl_proxy = "http://{}".format(proxy_auth)
    capabilities = webdriver.DesiredCapabilities.CHROME
    prox.add_to_capabilities(capabilities)
    # chrome_options.add_argument(f'--proxy-server={proxy_auth}')

    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options, desired_capabilities=capabilities)
    except Exception:
        driver = webdriver.Chrome(service=Service('driver/124.0.6367.207/chromedriver-win32/chromedriver.exe'), options=chrome_options, desired_capabilities=capabilities)

    return driver

In [99]:
def map_scraper_with_scrolls(jenis, jenis_table, filter_wilayah, driver, proxy):
    proxy_count = 0
    cek_proxy = ''

    while proxy_count < 61:
        # kalau ipynb
        if cek_proxy == 'Proxy gagal':
            driver = get_driver()
            print('Proxy baru')

        # kalau py
        # driver = get_driver()
        
        try:
            df_cari = create_new_df_cari(jenis_table, filter_wilayah)
            print(f'Ekspektasi jumlah query di cycle ini: {len(df_cari)}')
            
            for i in range(len(df_cari)):
                total_time = time.time()
                provinsi = df_cari.iloc[i].iloc[0]
                kota = df_cari.iloc[i].iloc[1]
                kecamatan = df_cari.iloc[i].iloc[2]
                kelurahan = df_cari.iloc[i].iloc[3]
                idcari = int(df_cari.iloc[i].iloc[5])
                cari = f'{jenis} in {kelurahan}, {kecamatan}, {kota}, {provinsi}'
                url_cari = create_search_link(cari, None, '', 18)

                driver.get(url_cari)

                try:
                    WebDriverWait(driver, 10).until(EC.title_contains("Google Maps"))
                    cek_proxy = ''
                except Exception:
                    cek_proxy = 'Proxy gagal'
                    break
            
                try:
                    divSideBar=driver.find_element(By.CSS_SELECTOR, "div[role='feed']")
                except Exception:
                    query_count += 1
                    print(f'Query {query_count}/{len(df_cari)} kosong kelurahan {kelurahan} kecamatan {kecamatan} kota {kota} provinsi {provinsi}')
                    print(f'Total waktu {time.time() - total_time}')
                    continue

                keepScrolling=True
                while keepScrolling:
                    divSideBar.send_keys(Keys.PAGE_DOWN)
                    div_html = driver.find_element(By.TAG_NAME, "html").get_attribute('outerHTML')

                    if "You've reached the end of the list." in div_html or 'Anda telah mencapai akhir daftar.' in div_html:
                        keepScrolling=False

                soup_cari = BeautifulSoup(driver.page_source, 'html.parser')
                targets = soup_cari.find("div", {'role': 'feed'}).find_all('div', {'class': False})[:-1]
                targets_no_ad = [div for div in targets if div.find('div', {'jsaction':True})]

                a = 1
                while True:
                    try:
                        nama = targets_no_ad[a].find_all("div", {'class':True})[0].find('a')['aria-label']

                        try:
                            jml_rating = int(targets_no_ad[a].find_all("div")[17].find_all("span")[4].text.strip()[1:-1].replace(',',''))
                        except:
                            jml_rating = 0

                        alamat = targets_no_ad[a].find_all('span', {'aria-hidden':'', 'aria-label':'', 'class':''})[3].text.strip()

                        try:
                            tag_google = [span for span in targets_no_ad[a].find_all('span', {'aria-label':'', 'aria-hidden':'', 'class':''}) if not span.find('span')][0].text.strip()
                        except:
                            tag_google = ''

                        coordinate = re.search(r'!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)', targets_no_ad[a].find_all("div")[0].find("a")['href'])
                        longlat = f'{coordinate.group(1)}, {coordinate.group(2)}'
                        updatetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                        try:
                            with sqlite3.connect('backend/data.db') as connection:
                                cursor = connection.cursor()
                                query = f'INSERT INTO {jenis_table} (NAMA, KOORDINAT, JML_RATING, ALAMAT, TAG_GOOGLE, KELURAHAN, KECAMATAN, KOTA, PROVINSI, TIPE, IDCARI, DATA_UPDATE) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
                                params = (nama, longlat, jml_rating, alamat, tag_google, kelurahan, kecamatan, kota, provinsi, jenis, idcari, updatetime)
                                cursor.execute(query, params)
                        except Exception as e:
                            print(f'Error occurred: {str(e)} on kelurahan {kelurahan} kecamatan {kecamatan} kota {kota} provinsi {provinsi} index {a}')

                        a += 1

                    except Exception:
                        break
                
                print(f'Query {i}/{len(df_cari)} {jenis} di kelurahan {kelurahan} kecamatan {kecamatan} kota {kota} provinsi {provinsi} selesai diinput sebanyak {a-1} data')
                print(f'Total waktu {time.time() - total_time}')

            if cek_proxy == 'Proxy gagal':
                print(f'Proxy {proxy_count} gagal, mencoba proxy selanjutnya')
                proxy_count += 1
                driver.close()
                break

        except Exception:
            pass

    if proxy_count > 60 and cek_proxy == 'Proxy gagal':    
        status = 'Seluruh proxy gagal'
        try:
            driver.close()
        except:
            pass
        print(status)

    status = f'Scrape {jenis} selesai'
    if cek_proxy != 'Proxy gagal':
        try:
            driver.close()
        except:
            pass
        
    print(status)

In [100]:
# TODO buat database jenis

In [101]:
# TODO loop keseluruhan kode berdasarkan database jenis

database = 'sqlite' # opsi sqlite atau mariadb
proxy = 'proxyscrape' # opsi kosong atau proxyscrape

query = 'restaurants'

filter_wilayah = {'PROPINSI': '',
                  'KOTA': '',
                  'KECAMATAN': 'PANCORAN',
                  'KELURAHAN': ''}

db_check(database, clean_table_name(query, filter_wilayah))
random_pos_check(database)

driver = get_driver() # driver pertama di luar function agar bisa close driver kalau manual interrupt di ipynb

# PILIH SALAH SATU
map_scraper(database, query, filter_wilayah, proxy)
# map_scraper_with_scrolls(database, query, filter_wilayah, driver, proxy)

DatabaseError: Execution failed on sql 'SELECT PROPINSI, KOTA, KECAMATAN, KELURAHAN, KODEPOS, ID AS IDCARI FROM randomized_pos WHERE IDCARI > 0 AND KECAMATAN = pancoran': no such column: pancoran

In [ ]:
# TODO buat async function agar bisa beberapa scraper sekaligus
    # TODO ubah cek iterasi dari id cari ke kolom penanda iterasi